If you are running this on Google Colab, you need to uncomment (remove the `#`) and execute the following lines to install the Pyomo package, the solver, and some helper tools. If you are running this on Binder or elsewhere (e.g. your own computer) you can ignore this.

In [1]:
# !pip install pyomo==6.4.1
# !apt install glpk-utils
# !pip install "git+https://github.com/sjpfenninger/sen1511.git#egg=sen1511utils&subdirectory=sen1511utils"

In [2]:
import pyomo.environ as pyo

from sen1511utils import summarise_results

# Assignment 1 - Linear Programming (LP)

## Group task 1

Consider two generating units supplying at least the system demand P_total,D=650 MW for the coming hour (extra power above the system demand will be used for storage, which is assumed to be unlimited). The unit cost function is characterized by the parameters provided in the table below:

| Unit | Generation costs (EUR/MWh) | P_min,G (MW) | P_max,G (MW) |
|:---|---:|---:|---:|
| 1 | 35 | 0 | 600 |
| 2 | 55 | 100 | 300 |

Tasks 4.a) through 4.c) are solved on paper.

### 4.d)

Solve the problem in Python, calculate dual variables, and compare your solutions here with the excercise above.

In [3]:
model = pyo.ConcreteModel(name = "Primal Problem")
model.dual = pyo.Suffix(direction=pyo.Suffix.IMPORT)

##
# 1. Decision variables
##

model.Unit1 = pyo.Var(domain=pyo.NonNegativeReals)
model.Unit2 = pyo.Var(domain=pyo.NonNegativeReals)

##
# 2. Objective function
##

model.profit = pyo.Objective(
    expr = 35 * model.Unit1 + 55 * model.Unit2,
    sense = pyo.minimize,
)

##
# 3. Constraints
##

model.demand = pyo.Constraint(expr = model.Unit1 + model.Unit2 >= 650)

model.UpperLimit1 = pyo.Constraint(expr = model.Unit1 <= 600)
model.LowerLimit1 = pyo.Constraint(expr = model.Unit1 >= 0)

model.UpperLimit2 = pyo.Constraint(expr = model.Unit2 <= 300)
model.LowerLimit2 = pyo.Constraint(expr = model.Unit2 >= 100)

# Solve the problem
solver = pyo.SolverFactory('glpk')
solver.solve(model)

{'Problem': [{'Name': 'unknown', 'Lower bound': 24750.0, 'Upper bound': 24750.0, 'Number of objectives': 1, 'Number of constraints': 6, 'Number of variables': 3, 'Number of nonzeros': 7, 'Sense': 'minimize'}], 'Solver': [{'Status': 'ok', 'Termination condition': 'optimal', 'Statistics': {'Branch and bound': {'Number of bounded subproblems': 0, 'Number of created subproblems': 0}}, 'Error rc': 0, 'Time': 0.007541656494140625}], 'Solution': [OrderedDict([('number of solutions', 0), ('number of solutions displayed', 0)])]}

In [4]:
summarise_results(model)

### 4.d)

Solve the dual problem including calculation of shadow prices in Python; what do you observe regarding the solution of the primal and the dual problem?

<div class="alert alert-block alert-info">

💡 We will call the dual model `dual_model` instead of `model`. This way, the original (primal) model is still available as well if we want to analyse it further.

</div>

In [5]:
dual_model = pyo.ConcreteModel(name = "Dual Problem")
dual_model.dual = pyo.Suffix(direction=pyo.Suffix.IMPORT)

##
# 1. Decision variables
##

dual_model.z1 = pyo.Var(domain=pyo.NonNegativeReals)
dual_model.z2 = pyo.Var(domain=pyo.NonNegativeReals)
dual_model.z3 = pyo.Var(domain=pyo.NonNegativeReals)
dual_model.z4 = pyo.Var(domain=pyo.NonNegativeReals)

##
# 2. Objective function
##

dual_model.profit = pyo.Objective(
    expr = 650*dual_model.z1 - 600*dual_model.z2 + 100*dual_model.z3 - 300*dual_model.z4,
    sense = pyo.maximize
)

##
# 3. Constraints
##

dual_model.c1 = pyo.Constraint(expr = dual_model.z1 - dual_model.z2 <= 35)
dual_model.c2 = pyo.Constraint(expr = dual_model.z1 + dual_model.z3 - dual_model.z4 <= 55)

# Solve the problem
solver = pyo.SolverFactory('glpk')
solver.solve(dual_model)

{'Problem': [{'Name': 'unknown', 'Lower bound': 24750.0, 'Upper bound': 24750.0, 'Number of objectives': 1, 'Number of constraints': 3, 'Number of variables': 5, 'Number of nonzeros': 6, 'Sense': 'maximize'}], 'Solver': [{'Status': 'ok', 'Termination condition': 'optimal', 'Statistics': {'Branch and bound': {'Number of bounded subproblems': 0, 'Number of created subproblems': 0}}, 'Error rc': 0, 'Time': 0.010075807571411133}], 'Solution': [OrderedDict([('number of solutions', 0), ('number of solutions displayed', 0)])]}

In [6]:
summarise_results(dual_model)

### 5.a)

<div class="alert alert-block alert-info">

💡 Here we are making yet another change to what variable name we give to the model. We simply use `m` which, thanks to its brevity, makes the rest of the model formulation easier to read (and faster to type).

</div>

In [7]:
m = pyo.ConcreteModel(name = "Model 5.a)")
m.dual = pyo.Suffix(direction=pyo.Suffix.IMPORT)

##
# 1. Decision variables
##

m.A1 = pyo.Var(domain=pyo.NonNegativeReals)
m.A2 = pyo.Var(domain=pyo.NonNegativeReals)
m.A3 = pyo.Var(domain=pyo.NonNegativeReals)
m.B1 = pyo.Var(domain=pyo.NonNegativeReals)
m.B2 = pyo.Var(domain=pyo.NonNegativeReals)
m.B3 = pyo.Var(domain=pyo.NonNegativeReals)

##
# 2. Objective function
##

m.profit = pyo.Objective(
    expr = 30 * m.A1 + 32 * m.A2 + 35 * m.A3 + 80 * m.B1 + 75 * m.B2 + 72 * m.B3,
    sense = pyo.minimize,
)

##
# 3. Constraints
##

m.A1_lower = pyo.Constraint(expr = m.A1 >= 50)
m.A1_upper = pyo.Constraint(expr = m.A1 <= 500)
m.A2_lower = pyo.Constraint(expr = m.A2 >= 50)
m.A2_upper = pyo.Constraint(expr = m.A2 <= 500)
m.A3_lower = pyo.Constraint(expr = m.A3 >= 40)
m.A3_upper = pyo.Constraint(expr = m.A3 <= 400)
m.B1_lower = pyo.Constraint(expr = m.B1 >= 40)
m.B1_upper = pyo.Constraint(expr = m.B1 <= 400)
m.B2_lower = pyo.Constraint(expr = m.B2 >= 40)
m.B2_upper = pyo.Constraint(expr = m.B2 <= 400)
m.B3_lower = pyo.Constraint(expr = m.B3 >= 50)
m.B3_upper = pyo.Constraint(expr = m.B3 <= 500)
m.demand = pyo.Constraint(expr = m.A1 + m.A2 + m.A3 + m.B1 + m.B2 + m.B3 == 1500)
m.emission_limit = pyo.Constraint(expr = 1200 * m.A1 + 1200 * m.A2 + 1200 * m.A3 + 800 * m.B1 + 800 * m.B2 + 800 * m.B3 <= 1400000)

# Solve the problem
solver = pyo.SolverFactory('glpk')
solver.solve(m)

{'Problem': [{'Name': 'unknown', 'Lower bound': 89300.0000000001, 'Upper bound': 89300.0000000001, 'Number of objectives': 1, 'Number of constraints': 15, 'Number of variables': 7, 'Number of nonzeros': 25, 'Sense': 'minimize'}], 'Solver': [{'Status': 'ok', 'Termination condition': 'optimal', 'Statistics': {'Branch and bound': {'Number of bounded subproblems': 0, 'Number of created subproblems': 0}}, 'Error rc': 0, 'Time': 0.008717060089111328}], 'Solution': [OrderedDict([('number of solutions', 0), ('number of solutions displayed', 0)])]}

In [8]:
summarise_results(m)

<div class="alert alert-block alert-info">

💡 To avoid having to manually specify all the upper and lower limits for the variables as separate constraints, you can also set the *variable bounds* during the creation of the variable as follows:

```python
m.A1 = pyo.Var(domain=pyo.NonNegativeReals, bounds=(50, 500))
m.A2 = pyo.Var(domain=pyo.NonNegativeReals, bounds=(50, 500))
m.A3 = pyo.Var(domain=pyo.NonNegativeReals, bounds=(40, 400))
m.B1 = pyo.Var(domain=pyo.NonNegativeReals, bounds=(40, 400))
m.B2 = pyo.Var(domain=pyo.NonNegativeReals, bounds=(40, 400))
m.B3 = pyo.Var(domain=pyo.NonNegativeReals, bounds=(50, 500))
```

The disadvantage is that you will not get the constraints you replace that way in the list of constrainst when analysing the model with `summarise_lp_results`.
    
</div>

### 5.b)

<div class="alert alert-block alert-info">

💡 We are overwriting the model from 5.a with that of 5.b by also assigning it to the variable `m`.

</div>

In [9]:
m = pyo.ConcreteModel(name = "Model 5.b)")
m.dual = pyo.Suffix(direction=pyo.Suffix.IMPORT)

##
# 1. Decision variables
##

m.A1 = pyo.Var(domain=pyo.NonNegativeReals)
m.A2 = pyo.Var(domain=pyo.NonNegativeReals)
m.A3 = pyo.Var(domain=pyo.NonNegativeReals)
m.B1 = pyo.Var(domain=pyo.NonNegativeReals)
m.B2 = pyo.Var(domain=pyo.NonNegativeReals)
m.B3 = pyo.Var(domain=pyo.NonNegativeReals)
m.C1 = pyo.Var(domain=pyo.NonNegativeReals)
m.C2 = pyo.Var(domain=pyo.NonNegativeReals)

##
# 2. Objective function
##

m.profit = pyo.Objective(
    expr = 30 * m.A1 + 32 * m.A2 + 35 * m.A3 + 80 * m.B1 + 75 * m.B2 + 72 * m.B3 + 90 * m.C1 + 85 * m.C2,
    sense = pyo.minimize,
)

##
# 3. Constraints
##

m.A1_lower = pyo.Constraint(expr = m.A1 >= 50)
m.A1_upper = pyo.Constraint(expr = m.A1 <= 500)
m.A2_lower = pyo.Constraint(expr = m.A2 >= 50)
m.A2_upper = pyo.Constraint(expr = m.A2 <= 500)
m.A3_lower = pyo.Constraint(expr = m.A3 >= 40)
m.A3_upper = pyo.Constraint(expr = m.A3 <= 400)
m.B1_lower = pyo.Constraint(expr = m.B1 >= 40)
m.B1_upper = pyo.Constraint(expr = m.B1 <= 400)
m.B2_lower = pyo.Constraint(expr = m.B2 >= 40)
m.B2_upper = pyo.Constraint(expr = m.B2 <= 400)
m.B3_lower = pyo.Constraint(expr = m.B3 >= 50)
m.B3_upper = pyo.Constraint(expr = m.B3 <= 500)
m.C1_lower = pyo.Constraint(expr = m.C1 >= 70)
m.C1_upper = pyo.Constraint(expr = m.C1 <= 700)
m.C2_lower = pyo.Constraint(expr = m.C2 >= 60)
m.C2_upper = pyo.Constraint(expr = m.C2 <= 600)
m.demand = pyo.Constraint(expr = m.A1 + m.A2 + m.A3 + m.B1 + m.B2 + m.B3 + m.C1 + m.C2 == 1500)
m.emission_limit = pyo.Constraint(
    expr = 1200 * m.A1 + 1200 * m.A2 + 1200 * m.A3 + 800 * m.B1 + 800 * m.B2 + 800 * m.B3 + 400 * m.C1 + 400 * m.C2 <= 1400000
)

# Solve the problem
solver = pyo.SolverFactory('glpk')
solver.solve(m)

{'Problem': [{'Name': 'unknown', 'Lower bound': 76165.0000000001, 'Upper bound': 76165.0000000001, 'Number of objectives': 1, 'Number of constraints': 19, 'Number of variables': 9, 'Number of nonzeros': 33, 'Sense': 'minimize'}], 'Solver': [{'Status': 'ok', 'Termination condition': 'optimal', 'Statistics': {'Branch and bound': {'Number of bounded subproblems': 0, 'Number of created subproblems': 0}}, 'Error rc': 0, 'Time': 0.008586883544921875}], 'Solution': [OrderedDict([('number of solutions', 0), ('number of solutions displayed', 0)])]}

In [10]:
summarise_results(m)

,Name,Value
0,profit,76165.000000
,Name,Value
0,A1,500.000000
1,A2,395.000000
2,A3,40.000000
3,B1,40.000000
4,B2,40.000000
5,B3,50.000000
6,C1,70.000000
7,C2,365.000000
